In [1]:
import torch
from PIL import Image
from torch import nn
from torchvision.transforms import transforms

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet , self).__init__() # 集成nn.Module的初始化方式
        self.model = nn.Sequential(
            # 每个样本input[3,120,120] output[48,29,29]  
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  #二维卷积(1) input[3, 224, 224]  output[48, 55, 55] 
            nn.ReLU(inplace=True), # 激活函数，对输出进行处理
            # output[48,14,14] 
            nn.MaxPool2d(kernel_size=3, stride=2),  #最大池化(1) output[48, 27, 27]
            # output[128,14,14]
            nn.Conv2d(48, 128, kernel_size=5, padding=2),  #二维卷积(2) output[128, 27, 27]
            nn.ReLU(inplace=True), # 激活函数，对输出进行处理
            # output[128,6,6]
            nn.MaxPool2d(kernel_size=3, stride=2),  #最大池化(2) output[128, 13, 13]
            # output[192,6,6]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),  #二维卷积(3) output[192, 13, 13]
            nn.ReLU(inplace=True),  # 激活函数，对输出进行处理
            # output[192,6,6]
            nn.Conv2d(192, 192, kernel_size=3, padding=1),  #二维卷积(4) output[192, 13, 13]
            nn.ReLU(inplace=True), # 激活函数，对输出进行处理
            # output[128,6,6]
            nn.Conv2d(192, 128, kernel_size=3, padding=1),  #二维卷积(5) output[128, 13, 13]
            nn.ReLU(inplace=True), # 激活函数，对输出进行处理
            # output[128,2,2]
            nn.MaxPool2d(kernel_size=3, stride=2),  #最大池化(3) output[128, 6, 6]
            # output[128*2*2]
            nn.Flatten(),  # 展平，即降维成一维, output[128*6*6]
            nn.Dropout(p=0.5), # 随机丢失，概率为0.5
            # output[2048]
            nn.Linear(512, 2048), # 全连接层(1) [128*6*6,2048] 
            nn.ReLU(inplace=True), # 激活函数，对输出进行处理
            nn.Dropout(p=0.5), # 随机丢失，概率为0.5
            # output[1024]
            nn.Linear(2048, 1024),  # 全连接层(2) [2048,1024] 
            nn.ReLU(inplace=True),  # 激活函数，对输出进行处理
            # output[7]
            nn.Linear(1024, 2), # 全连接层(2) [1024,1000] 

        )
    def forward(self , x): # 前向传播
        x = self.model(x)
        return x


e:\python\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'E:\python\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [8]:
# image_path = "./data/test/1.JPG"#需要测试的图片放入当前文件夹下，这里改成自己的图片名即可
image_path = "./data/test/4.JPG"
trans = transforms.Compose([transforms.Resize((120 , 120)),
                           transforms.ToTensor()])
image = Image.open(image_path)

image = image.convert("RGB")
image = trans(image)

image = torch.unsqueeze(image, dim=0)

classes = ["cat" , "dog"] # 分类

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))


alexnet = AlexNet()
# 将训练得到的alexnet模型载入
alexnet.load_state_dict(torch.load("alexnet.pth", map_location=device))

alexnet.eval() # 屏蔽dropout层
outputs = alexnet(image)

print((outputs.max(0)[0]))

ans = (outputs.argmax(1)).item()
print(ans)
print('结果为',classes[ans])

using cuda:0 device.
tensor([ 0.0987, -0.0478], grad_fn=<MaxBackward0>)
0
结果为 cat
